In [2]:
import pandas as pd

initial_data = pd.read_csv('Taxi_Trips.csv')
initial_data.head(10)
    

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,0287f53fedcde6b0af9aab9e32cbd2cadb337eaa,e54db25f18193a08f1f5754515e8c338480e04fb938ed3...,03/01/2024 12:00:00 AM,03/01/2024 12:00:00 AM,15.0,0.09,NaN,NaN,8.0,8.0,...,35.0,38.75,Credit Card,City Service,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
1,1f0034299b914fd7b45002568576a353c83acb2f,4a263f78f56815be4acd38658af8fc1824ce4e15f7ec81...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,900.0,3.00,NaN,NaN,24.0,8.0,...,0.0,12.00,Cash,Taxi Affiliation Services,41.901207,-87.676356,POINT (-87.6763559892 41.9012069941),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
2,01a0b77b722a0b91f45cb4fc90b3e64f76fd0681,4f1e94982e6851725add382f7981d64006ae7c38f3664f...,03/01/2024 12:00:00 AM,03/01/2024 12:00:00 AM,711.0,5.84,NaN,NaN,56.0,NaN,...,5.0,26.70,Credit Card,City Service,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603),NaN,NaN,NaN
3,14aff2071ac81c6450a8f8b0e1534497da900197,de8e8659ceb9eb0da842a46b60c0a5207098ac69fc23ab...,03/01/2024 12:00:00 AM,03/01/2024 12:30:00 AM,1770.0,13.36,NaN,NaN,76.0,3.0,...,4.0,47.10,Credit Card,Sun Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
4,16d3a633e2f380cee4a7fa4677db163b6e91255d,d79d3e19a1d5f6cb0cd4449d5579412262a2f1b182799d...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,849.0,6.13,NaN,NaN,8.0,3.0,...,0.0,22.82,Mobile,5 Star Taxi,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
5,07a1e0cf2a3e0def140837ebda3d4f1befd3a4fd,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,03/01/2024 12:00:00 AM,03/01/2024 12:30:00 AM,2280.0,2.00,1.703198e+10,NaN,76.0,NaN,...,46.0,154.50,Credit Card,Taxi Affiliation Services,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),NaN,NaN,NaN
6,4d61a32dfe8fc947843672ca916c7ecffb91f4dc,354fafd255c37a15b0c8f6f67d810b3e5374f54ebbadb9...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,1416.0,11.17,NaN,NaN,28.0,28.0,...,0.0,29.82,Prcard,5 Star Taxi,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.874005,-87.663518,POINT (-87.6635175498 41.874005383)
7,cce9346b4b5af0d65bbbf042b2ab93957b6abb77,9d2bc650f24375604a82a15892dfbbea47dce34e8c3236...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,1553.0,13.21,NaN,NaN,76.0,5.0,...,4.0,46.20,Credit Card,City Service,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.947792,-87.683835,POINT (-87.6838349425 41.9477915865)
8,76a403a0963ffdb0f999b5a3b7c303c30169a961,3c814d3baedca9be4de8ddb5547c7dec404a08e148740e...,03/01/2024 12:00:00 AM,03/01/2024 12:30:00 AM,1755.0,12.52,NaN,NaN,76.0,4.0,...,4.0,48.43,Credit Card,Sun Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.975171,-87.687516,POINT (-87.6875155152 41.9751709433)
9,6bf67f3a4636d6d99b25ba86d2e6d3b28235b04b,791d74df896226a452a8e223e2ec9fa0df7d80bb7ca180...,03/01/2024 12:00:00 AM,03/01/2024 12:15:00 AM,1260.0,9.90,NaN,NaN,28.0,6.0,...,0.0,26.75,Cash,Taxi Affiliation Services,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014)
